In [20]:
import pandas as pd

def read_investment_log(file_path):
    investments = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split(', ')
            id_part = parts[0].split(': ')[1]
            target_part = parts[1].split(': ')[1]
            amount_part = int(parts[2].split(': ')[1])
            investments.append({
                '투자자': id_part,
                '투자 대상': target_part,
                '투자 금액': amount_part
            })
    
    return investments

def aggregate_investments(investments):
    df = pd.DataFrame(investments)
    
    # 투자 대상별로 총 투자 금액을 구함
    total_investment = df.groupby('투자 대상')['투자 금액'].sum().reset_index().rename(columns={'투자 금액': '총 투자 금액'})
    
    # 각 투자 대상별로 투자자와 투자 금액을 문자열로 연결
    investor_details = df.groupby(['투자 대상', '투자자'])['투자 금액'].sum().reset_index()
    investor_details['투자 내역'] = investor_details.apply(lambda x: f"{x['투자자']} : {x['투자 금액']}", axis=1)
    
    # 투자 대상별로 투자 내역을 하나의 문자열로 결합
    detailed_investment = investor_details.groupby('투자 대상')['투자 내역'].apply(', '.join).reset_index()
    
    # 총 투자 금액 데이터프레임과 투자 내역 데이터프레임을 병합
    result = pd.merge(total_investment, detailed_investment, on='투자 대상')
    
    return result

# 사용 예시
file_path = 'investment_log.txt'
investments = read_investment_log(file_path)
aggregated_investments = aggregate_investments(investments)

# 결과 출력
print(aggregated_investments)


  투자 대상  총 투자 금액                                              투자 내역
0   구도윤     1600  구도윤 : 400, 김강현 : 300, 김대성 : 100, 임도원 : 100, 전해...
1   권태희      700                               구도윤 : 300, 임도원 : 400
2   김강현     2050  권태희 : 500, 김동우 : 600, 이도형 : 400, 전해성 : 250, 홍길...
3   김대성      950                               김강현 : 700, 전해성 : 250
4   서진영      800                    김동우 : 200, 이도형 : 300, 전해성 : 300
5   이도형      700         구도윤 : 300, 권태희 : 100, 김대성 : 100, 임도원 : 200
6   임도원     1000         권태희 : 100, 김대성 : 100, 이도형 : 300, 홍길동 : 500
7   전해성     2000         김대성 : 700, 김동우 : 200, 임도원 : 300, 홍길동 : 800
8   한민우      300                                          권태희 : 300


In [1]:
def parse_investment_file(file_path):
    investments = {}

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:
                parts = line.split(',')
                if len(parts) == 3:  # 세 개의 부분이 제대로 있는지 확인
                    id_part, target_part, amount_part = parts

                    user_id = id_part.split(':')[1].strip()
                    target = target_part.split(':')[1].strip()
                    amount = int(amount_part.split(':')[1].strip())

                    if target not in investments:
                        investments[target] = []

                    investments[target].append({
                        'investor': user_id,
                        'amount': amount
                    })
                else:
                    print(f"잘못된 형식의 줄이 있습니다: {line}")
    return investments


def calculate_total_investments(investment_data):
    total_investments = {}
    
    for target, investments in investment_data.items():
        total_investment = sum(investment['amount'] for investment in investments)
        total_investments[target] = total_investment

    return total_investments

def assign_teams(investment_data, team_count):
    total_investments = calculate_total_investments(investment_data)
    
    # 모든 투자자를 집합으로 모아 중복 없이 인원수 계산
    all_investors = set(investor for inv_list in investment_data.values() for investor in [x['investor'] for x in inv_list])
    total_people = len(all_investors)
    
    # 팀의 최대 인원 계산
    max_team_size = total_people // team_count
    remainder = total_people % team_count
    
    # 투자 합계가 높은 순서대로 팀장 선정
    sorted_leaders = sorted(total_investments.keys(), key=lambda k: total_investments[k], reverse=True)
    
    # 입력받은 팀 수만큼 팀장 선출
    leaders = sorted_leaders[:team_count]
    teams = {leader: [] for leader in leaders}

    # 팀에 합류시키기
    for leader in leaders:
        # 해당 팀장에게 투자한 사람들 목록
        investments = sorted(investment_data[leader], key=lambda x: x['amount'], reverse=True)
        for investor_data in investments:
            investor = investor_data['investor']
            # 팀장을 팀원에서 제외하고, 이미 다른 팀에 소속되지 않았고, 팀 최대 인원 수를 넘지 않도록 확인
            if investor not in leaders and not any(investor in team for team in teams.values()) and len(teams[leader]) < max_team_size:
                teams[leader].append(investor)
    
    # 남은 인원 확인 및 배분
    assigned_investors = set(investor for team in teams.values() for investor in team)
    remaining = list(all_investors - assigned_investors - set(leaders))
    
    if remaining:
        # 나머지 인원 배분
        for investor in remaining:
            for leader in leaders:
                if len(teams[leader]) < max_team_size + (1 if remainder > 0 else 0):
                    teams[leader].append(investor)
                    remainder -= 1
                    break

    return teams



# 텍스트 데이터 파일에서 읽어오기
file_path = 'investment_log.txt'
investment_data = parse_investment_file(file_path)

# 팀 총 수를 입력받음
team_count = int(input("팀 총 수를 입력하세요: "))

# 팀 할당
teams = assign_teams(investment_data, team_count)

# 결과 출력
print("\n팀 구성 결과:")
for team, members in teams.items():
    print(f"{team} 팀: {', '.join(members)}")



팀 구성 결과:
서진영 팀: 임관훈, 오경빈, 김동우, 신채범
구도윤 팀: 이도형, 전해성, 김대성
권태희 팀: 


In [4]:
from collections import defaultdict

def calculate_investment_sums(investment_data):
    investment_sums = defaultdict(int)
    
    for person,  in investment_data.items():
        for investment in investments:
            investment_sums[person] += investment['amount']
    
    sorted_investments = sorted(investment_sums.items(), key=lambda x: x[1], reverse=True)
    
    return {name: amount for name, amount in sorted_investments}


In [5]:
result = calculate_investment_sums(investment_data)
result

{'서진영': 2700,
 '구도윤': 1950,
 '권태희': 1800,
 '전해성': 1350,
 '김대성': 1250,
 '김동우': 1150,
 '임관훈': 950,
 '신채범': 900,
 '오경빈': 750,
 '이도형': 750}